In [ ]:
using Plots,Random,Measures,CSV,DataFrames,Statistics,Distributions
gr();

include("auxilliary.jl"); include("poimcmc.jl");

## Inputs

In [ ]:
nsmp = 200000
myrng = MersenneTwister(); 

In [ ]:
# Load the fixed enter times for infecteds from an Enter/Exit 
df = CSV.read("MCMCsmp.csv",DataFrame,header=false); 
df = df[:,1:2]; rename!(df,["Column1"=>"name","Column2"=>"value"]); df[!,"name"].=Symbol.(df[!,"name"]);
prm,vkeys = rdprm(df[!,"value"],df[!,"name"]); nmax=Int64(prm[:nmax]);
println("Dust measurement:")
prm[:Y]

In [ ]:
# Load the β-posterior
dfβ = CSV.read("abpost.csv",DataFrame,header=false);
βpost=convert(Vector{Float64},dfβ[2,2:end]);

In [ ]:
βhist = rand(myrng,βpost,nsmp);
Ahist = [rand(myrng,Gamma(1.0,1/βhist[i])) for i=1:nsmp];
Aₓhist = rand(myrng,Normal(prm[:Aₓμ],prm[:Aₓσ]),nsmp);
Lhist = Aₓhist + rand(myrng,Normal(prm[:Lμ],prm[:Lσ]),nsmp);

p1 = histogram(βhist,title="β",labels="",normalize=:pdf,bins=75);
p2 = histogram(Ahist,title="A",labels="",normalize=:pdf,bins=75);
p3 = histogram(Aₓhist,title="Aₓ",labels="",normalize=:pdf,bins=75);
p4 = histogram(Lhist,title="L",labels="",normalize=:pdf,bins=75);

lay = @layout [a b;c d]
plot(p1,p2,p3,p4)

## Sample dust values

In [ ]:
Y = Vector{Float64}(undef,nsmp);
μλ = Vector{Float64}(undef,nsmp);
μp = 0.0; gen=[1,0]; #nsmp x nmax
@inbounds for k=1:nsmp*nmax
    if gen[2]<nmax
        gen[2]+=1;
    else
        gen[1]+=1;gen[2]=1;
    end
    i=gen[1]; j=gen[2];
    
    # Set base Poisson mean if needed
    if j==1
        μp = 0.0;
    end
    
    # Sample the infection time
    tj = Symbol(:t,j); tej = Symbol(:te,j); tℓj = Symbol(:tℓ,j);
    prm[tj] = prm[tej]-10+10*rand(myrng);
    
    # Sample the amplitude 
    Aj = Symbol(:A,j);
    β = rand(myrng,βpost);
    Γdistr = Gamma(1.0,1/β);
    prm[Aj] = rand(myrng,Γdistr);
    
    # Sample the position of the peak
    Axj = Symbol(:Aₓ,j);
    Ndistr = Normal(prm[:Aₓμ],prm[:Aₓσ]);
    prm[Axj] = rand(myrng,Ndistr);
    
    # Sample the shedding duration conditional on the peak
    Lj = Symbol(:L,j);
    Ndistr = Normal(prm[:Lμ],prm[:Lσ]);
    prm[Lj] = prm[Axj] + rand(myrng,Ndistr);
    
    # Compute contribution to Poisson mean
    μp += shedλ(prm[Aj],prm[Lj],prm[tj],prm[:T],prm[Axj],prm[:ξ],prm[tej],prm[tℓj]);
    
    if j==nmax
        μλ[i] = μp;
        Pdistr = Poisson(μp);
        Y[i] = rand(myrng,Pdistr);
    end
end 

In [ ]:
p1 = histogram(Y,title="Observed Dust",xlabel="cp/mg",ylabel="density",normalize=:pdf,labels="",bins=75);
p2 = histogram(μλ,title="Shedding Mean",xlabel="cp/mg",ylabel="density",normalize=:pdf,labels="",bins=75);
lay = @layout [a b];
plot(p1,p2,layout=lay,size=(900,300),margin=5mm)